In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE

In [16]:
## Very very basic data ingest

ds_tr = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
#print(ds_tr.info())
ds_pr = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')
#print(ds_pr.info())
dataset = pd.concat([ds_tr, ds_pr], axis=0)
#print(dataset.info())

cat_col = []
num_col = []
for col in ds_tr.columns:
    if np.dtype(ds_tr[col]) == 'object':
        cat_col.append(col)

print(f'Categorical is {cat_col}')

#dataset = pd.get_dummies(dataset, drop_first=True)
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for cat in cat_col:
    dataset[cat] = encoder.fit_transform(dataset[cat])

ds_train = dataset.loc[dataset['target'].notnull(),:]
ds_predict = dataset.loc[dataset['target'].isnull(),:]

y_train = ds_train['target']
X_train = ds_train.drop(['id','target'], axis=1)

id = ds_predict['id']
ds_predict = ds_predict.drop(['id','target'], axis=1)

Categorical is ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18']


In [4]:
## Visualize dataset
# In categorical, use histogram
'''
fig, axes = plt.subplots(5,4, figsize=(20,10))
for i, cat in enumerate(cat_col):
    row = int(i/4)
    col = int(i%4)
    axes[row, col].set_title(cat)
    plt.tight_layout()
    ds_tr[cat].hist(ax=axes[row, col])

# In numerical, use scatter
fig, axes = plt.subplots(3,4, figsize=(20,10))
for i, num in enumerate(num_col):
    row = int(i/4)
    col = int(i%4)
    axes[row, col].set_title(num)
    plt.tight_layout()
    ds_tr[num].plot(ax=axes[row, col])
'''

'\nfig, axes = plt.subplots(5,4, figsize=(20,10))\nfor i, cat in enumerate(cat_col):\n    row = int(i/4)\n    col = int(i%4)\n    axes[row, col].set_title(cat)\n    plt.tight_layout()\n    ds_tr[cat].hist(ax=axes[row, col])\n\n# In numerical, use scatter\nfig, axes = plt.subplots(3,4, figsize=(20,10))\nfor i, num in enumerate(num_col):\n    row = int(i/4)\n    col = int(i%4)\n    axes[row, col].set_title(num)\n    plt.tight_layout()\n    ds_tr[num].plot(ax=axes[row, col])\n'

# SMOTE 
To resolve imbalanced data, try [SMOTENC](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTENC.html).

In [17]:
print(f'Orignal X shape = {X_train.shape}')
sm = SMOTE(k_neighbors=5, n_jobs=-1, random_state=42)
X_train, y_train = sm.fit_resample(X_train,y_train)
print(f'SMOTED X shape = {X_train.shape}')

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

Orignal X shape = (300000, 30)
SMOTED X shape = (441078, 30)


# Lightgbm Classification

In [ ]:
## LightGBM Classification

import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score

def objective(trial):
    params = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'n_estimators': 1000,
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
        }
    
    lgb_classifier = lgb.LGBMClassifier(**params)
    lgb_classifier.fit(X_tr, y_tr, eval_set=[(X_te, y_te)],  verbose=False, early_stopping_rounds=50)
    y_pred = lgb_classifier.predict(X_te)
    labels = np.rint(y_pred)
    auc_roc_score = roc_auc_score(y_te,labels)
    return auc_roc_score
       
lgb_train = lgb.Dataset(X_tr, y_tr)
lgb_test = lgb.Dataset(X_te, y_te)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=-1)
lgb_best = study.best_params
print(lgb_best)

[I 2021-03-24 06:46:50,419] A new study created in memory with name: no-name-86b5cc16-669b-497a-a00f-db1a2e33ccf1
[I 2021-03-24 06:49:57,355] Trial 1 finished with value: 0.8540438728297047 and parameters: {'lambda_l1': 0.5515432756730366, 'lambda_l2': 2.3013063191068924e-08, 'num_leaves': 54, 'feature_fraction': 0.9511299826727279, 'bagging_fraction': 0.5452644728003322, 'bagging_freq': 6, 'min_child_samples': 71, 'learning_rate': 0.00408887987469744}. Best is trial 1 with value: 0.8540438728297047.
[I 2021-03-24 06:50:38,547] Trial 2 finished with value: 0.8358967829757635 and parameters: {'lambda_l1': 5.393347141761578e-05, 'lambda_l2': 7.329239153803771e-06, 'num_leaves': 60, 'feature_fraction': 0.8077556067481473, 'bagging_fraction': 0.8410911412556719, 'bagging_freq': 5, 'min_child_samples': 76, 'learning_rate': 0.0004136894145158887}. Best is trial 1 with value: 0.8540438728297047.
[I 2021-03-24 06:51:51,759] Trial 3 finished with value: 0.8428027755183723 and parameters: {'lamb

[I 2021-03-24 07:16:10,284] Trial 20 finished with value: 0.850236035257072 and parameters: {'lambda_l1': 4.811132125189796, 'lambda_l2': 0.4911422447521465, 'num_leaves': 218, 'feature_fraction': 0.7737160869327284, 'bagging_fraction': 0.8716491357125524, 'bagging_freq': 2, 'min_child_samples': 96, 'learning_rate': 0.0007238581615955154}. Best is trial 14 with value: 0.8751869650362256.
[I 2021-03-24 07:18:38,254] Trial 21 finished with value: 0.8499640692915683 and parameters: {'lambda_l1': 2.8084087482102874e-06, 'lambda_l2': 0.2977765164765415, 'num_leaves': 215, 'feature_fraction': 0.7582036899465079, 'bagging_fraction': 0.8802367233174714, 'bagging_freq': 2, 'min_child_samples': 100, 'learning_rate': 0.0005788590220972139}. Best is trial 14 with value: 0.8751869650362256.
[I 2021-03-24 07:21:06,157] Trial 22 finished with value: 0.8453163926098671 and parameters: {'lambda_l1': 4.196579609506097e-06, 'lambda_l2': 0.18158662847735724, 'num_leaves': 212, 'feature_fraction': 0.890394

In [ ]:
lgb_best['metric'] = 'roc_auc_score'

Estimators=[80,100,120,150,180,200,210,220,250,280,300,350,400,500,800,1000,1200,1500,2000]
optimized_est = 0
auc_status = 0
for i in Estimators:
    lgb_best['n_estimators']=i
    LGBM = LGBMClassifier(**lgb_best)
    LGBM.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False, early_stopping_rounds=50)
    y_pred = LGBM.predict_proba(X_val)[:,1]
    auc = roc_auc_score(y_te, y_pred)
    if auc > auc_status:
        auc_status = auc
        optimized_est = i
    print("\n\n For Estimators = {} \t  AUC Score : {} \n\n".format(i,auc))

print(f'Best estimator = {optimized_est}, auc = {auc_status}')
lgb_best['n_estimators'] = optimized_est

# CatBoost Classification
* category columnsをfit時に選択すると、時間むっちゃかかる
* SMOTENC + CatBoostでは、 "Best is trial 45 with value: 0.13424544780540504." の程度

In [7]:
## CatBoost Classification
'''
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

def objective(trial):
    params = {
        #'iterations' : trial.suggest_int('iterations', 50, 3000),
        'iterations' : 1000,
        'loss_function': 'CrossEntropy',
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-6, 0.1),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50)
    }

    # 学習
    model = CatBoostClassifier(**params)
    
    model.fit(X=X_tr, y=y_tr, eval_set=[(X_te, y_te)], use_best_model=True, early_stopping_rounds=50, verbose=False)
    y_pred = model.predict(X_te)
    
    return 1 - roc_auc_score(y_te, y_pred)
    
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3)

study = optuna.create_study()
study.optimize(objective, n_trials=100, n_jobs=-1)
cat_best = study.best_params
print(cat_best)
'''

"\nimport optuna\nfrom catboost import CatBoostClassifier\nfrom sklearn.metrics import roc_auc_score\n\ndef objective(trial):\n    params = {\n        #'iterations' : trial.suggest_int('iterations', 50, 3000),\n        'iterations' : 1000,\n        'loss_function': 'CrossEntropy',\n        'depth' : trial.suggest_int('depth', 4, 10),                                       \n        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-6, 0.1),               \n        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       \n        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), \n        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),\n        'od_wait' :trial.suggest_int('od_wait', 10, 50)\n    }\n\n    # 学習\n    model = CatBoostClassifier(**params)\n    \n    model.fit(X=X_tr, y=y_tr, eval_set=[(X_te, y_te)], use_best_model=True, early_stopping_rounds=50, verbose=False)\n    y_pred = 

In [ ]:
# Best params
# lgb_best = {'lambda_l1': 3.718273500796517e-07, 'lambda_l2': 3.861811885251849e-05, 'num_leaves': 172, 'feature_fraction': 0.6647133561508157, 'bagging_fraction': 0.7033334859709666, 'bagging_freq': 3, 'min_child_samples': 76, 'learning_rate': 0.009732761557836887}

In [10]:
## Predict
lgb_classifier = lgb.LGBMClassifier(**lgb_best)
lgb_classifier.fit(X=X_tr, y=y_tr, eval_set=[(X_te, y_te)], verbose=False)
y_pred = lgb_classifier.predict_proba(ds_predict, num_iteration=lgb_classifier.best_iteration_)[:,1]

'''
predictor = CatBoostClassifier(**cat_best)
predictor.fit(X=X_tr, y=y_tr, eval_set=[(X_te, y_te)],cat_features=cat_col, 
               use_best_model=True, early_stopping_rounds=50, verbose=False)
    
y_pred = predictor(predict_ds)
'''


output = pd.DataFrame({'id': id, 'target': y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
